In [3]:
import string
import pandas as pd
import glob
import numpy as np
import nltk
import matplotlib.pyplot as plt
import re
import seaborn as sns
import contractions
import json

from autocorrect import Speller
from nltk.corpus import stopwords, words
from nltk.tokenize import word_tokenize
from string import punctuation
from datetime import datetime
from sklearn.metrics import r2_score
from collections import Counter
from readability import Readability

In [5]:
path = 'TeacherStudentChatroomCorpus_v2/public'
filenames = glob.glob(path+"/*.tsv")

In [24]:
img_names = glob.glob('plots/student'+"/*")
img_names = ['figs/student/'+name.split('\\')[1] for name in img_names]
img_names

['figs/student/student002_MLU_AoA.png',
 'figs/student/student002_TTR_utterances_words_scaffolding_proportion.png',
 'figs/student/student002_WF.png',
 'figs/student/student003_MLU_AoA.png',
 'figs/student/student003_TTR_utterances_words_scaffolding_proportion.png',
 'figs/student/student003_WF.png',
 'figs/student/student004_MLU_AoA.png',
 'figs/student/student004_TTR_utterances_words_scaffolding_proportion.png',
 'figs/student/student004_WF.png',
 'figs/student/student005_MLU_AoA.png',
 'figs/student/student005_TTR_utterances_words_scaffolding_proportion.png',
 'figs/student/student005_WF.png',
 'figs/student/student006_MLU_AoA.png',
 'figs/student/student006_TTR_utterances_words_scaffolding_proportion.png',
 'figs/student/student006_WF.png',
 'figs/student/student007_MLU_AoA.png',
 'figs/student/student007_TTR_utterances_words_scaffolding_proportion.png',
 'figs/student/student007_WF.png',
 'figs/student/student009_MLU_AoA.png',
 'figs/student/student009_TTR_utterances_words_scaffol

In [34]:
latex_code = ""

for image_name in img_names:
    student_id = image_name.split("_")[0]
    aspect = ' '.join(image_name.split("_")[1:]).split('.')[0]

    latex_code += r"""\begin{subfigure}{0.3\textwidth} 
    \includegraphics[width=\textwidth]{""" + image_name + r"""} 
    \caption{""" + student_id + " " + aspect +  r"""} 
    \end{subfigure} 
    \hfill 
    """

print(latex_code)

\begin{subfigure}{0.3\textwidth} 
    \includegraphics[width=\textwidth]{figs/student/student002_MLU_AoA.png} 
    \caption{figs/student/student002 MLU AoA} 
    \end{subfigure} 
    \hfill 
    \begin{subfigure}{0.3\textwidth} 
    \includegraphics[width=\textwidth]{figs/student/student002_TTR_utterances_words_scaffolding_proportion.png} 
    \caption{figs/student/student002 TTR utterances words scaffolding proportion} 
    \end{subfigure} 
    \hfill 
    \begin{subfigure}{0.3\textwidth} 
    \includegraphics[width=\textwidth]{figs/student/student002_WF.png} 
    \caption{figs/student/student002 WF} 
    \end{subfigure} 
    \hfill 
    \begin{subfigure}{0.3\textwidth} 
    \includegraphics[width=\textwidth]{figs/student/student003_MLU_AoA.png} 
    \caption{figs/student/student003 MLU AoA} 
    \end{subfigure} 
    \hfill 
    \begin{subfigure}{0.3\textwidth} 
    \includegraphics[width=\textwidth]{figs/student/student003_TTR_utterances_words_scaffolding_proportion.png} 
    \captio

In [3]:
chatrooms = []
for filename in filenames:
    chatrooms.append(pd.read_csv(filename, sep='\t'))

chatrooms_all = pd.concat(chatrooms, axis=0, ignore_index=True)
chatrooms[0]

,timestamp,user.id,role,turn.number,anonymised,edited,responding.to,sequence,seq.type,focus,resource,assessment,nWords
0,2019-11-17 12:00:08,teacher001,teacher,1,"Hi <STUDENT>, hope I didn't get you up too early!","Hi <STUDENT>, hope I didn't get you up too early!",NaN,1.0,opening,NaN,NaN,NaN,10
1,2019-11-17 12:01:08,student002,student,2,"Don't worry, my exam is on next Saturday, so I...","Don't worry, my exam is next Saturday, so I sh...",NaN,NaN,NaN,NaN,NaN,NaN,14
2,2019-11-17 12:01:37,teacher001,teacher,3,"Ah OK, so good practice then...is that an IELT...","Ah OK, so good practice then...is that an IELT...",NaN,2.0,topic opening,NaN,NaN,NaN,10
3,2019-11-17 12:01:52,student002,student,4,Exactly.,Exactly.,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2019-11-17 12:02:11,teacher001,teacher,5,I've lost track of how many you've done,I've lost track of how many you've done,NaN,NaN,NaN,NaN,NaN,NaN,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,2019-11-17 13:01:33,teacher001,teacher,88,OK thank's <STUDENT> -we'll leave it there ok?...,OK thank's <STUDENT> -we'll leave it there ok?...,NaN,9.0,closing,NaN,NaN,NaN,14
88,2019-11-17 13:01:51,student002,student,89,"Okay, thank you!!","Okay, thank you!!",NaN,NaN,NaN,NaN,NaN,NaN,3
89,2019-11-17 13:02:08,teacher001,teacher,90,I'll be in touch - feel free to send me an ema...,I'll be in touch - feel free to send me an ema...,NaN,NaN,NaN,NaN,NaN,NaN,13
90,2019-11-17 13:02:27,student002,student,91,"Sure, see you.","Sure, see you.",NaN,NaN,NaN,NaN,NaN,NaN,3


In [6]:
metadata = pd.read_csv(path+"/teacherStudentChatroomCorpusPublicMetadata.csv")
metadata

,chat.num,filename,start.time,teacher,student,n.turns,n.teacher.turns,n.student.turns,n.words,n.teacher.words,n.student.words,student.cefr.level,student.age,student.L1,student.immersed.in.english,is_sett_annotated
0,2,teacherstudentchat00002.tsv,2019-11-17 12:00:08,teacher001,student002,92,52,40,1067,767,300,B1,22,Japanese,NaN,True
1,3,teacherstudentchat00003.tsv,2019-12-03 13:00:03,teacher001,student004,115,55,60,1623,943,680,C1,40,Spanish,NaN,True
2,4,teacherstudentchat00004.tsv,2019-12-03 15:29:58,teacher001,student003,96,51,45,1320,965,355,B2,32,Japanese,NaN,True
3,5,teacherstudentchat00005.tsv,2019-12-04 14:01:25,teacher001,student002,75,43,32,1121,936,185,B1,22,Japanese,NaN,True
4,6,teacherstudentchat00006.tsv,2019-12-09 13:00:35,teacher001,student004,138,75,63,1820,1109,711,C1,40,Spanish,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,257,teacherstudentchat00257.tsv,2021-06-10 23:00:00,teacher008,student009,145,69,76,1688,1099,589,B2,12,Ukrainian,n,False
256,258,teacherstudentchat00258.tsv,2021-06-15 11:01:30,teacher008,student014,103,59,44,1659,975,684,C2,30,Italian,n,False
257,259,teacherstudentchat00259.tsv,2021-06-17 12:17:50,teacher008,student012,150,75,75,1201,738,463,C2,33,Russian,n,False
258,260,teacherstudentchat00260.tsv,2021-06-18 09:38:19,teacher008,student011,183,77,106,1318,744,574,B2,26,"Russian,Ukrainian",n,False


In [5]:
seq_types = chatrooms_all['seq.type']
seq_types.dropna(inplace=True)
seq_types = seq_types.unique()
unique_seq_types=[]
for seq_type in seq_types:
    for type_split in seq_type.split(','):
        unique_seq_types.append(type_split)
unique_seq_types = set(unique_seq_types)
unique_seq_types

{'admin',
 'clarification',
 'closing',
 'disruption',
 'eliciting',
 'enquiry',
 'exercise',
 'free practice',
 'homework',
 'non-English',
 'opening',
 'presentation',
 'recap',
 'redirection',
 'reference',
 'repair',
 'revision',
 'scaffolding',
 'topic closure',
 'topic development',
 'topic opening'}

In [7]:
student_group = metadata.groupby("student")

In [20]:
[int(item) for item  in student_group['chat.num'].describe()['count'].values]

[3, 4, 28, 6, 4, 32, 48, 14, 51, 26, 26, 18]

In [7]:
student007_chats = []
for chat in chatrooms:
    for id in chat['user.id']:
        if id == 'student007':
            student007_chats.append(chat)
            break
chatC1 = pd.read_csv(path+"/teacherstudentchat00049.tsv", sep='\t')

In [8]:
l1_groups = metadata.groupby('student.L1')

Split the statistics by students' abilities (measured by CEFR for now)


In [9]:
cefr_groups_meta = metadata.groupby('student.cefr.level')
# chat number-2 = index in the chatrooms list

cefr_groups = {}
for group, contents in cefr_groups_meta:
    cefr_groups[group] = []
    for chat_num in contents['chat.num']:
        cefr_groups[group].append(chatrooms[chat_num-2])
    print(f'{group}: {len(cefr_groups[group])}')

B1: 35
B2: 144
C1: 29
C2: 52


In [10]:
AoA_list = pd.read_excel('AoA/AoA_51715_words.xlsx')[['Word', 'AoA_Kup_lem']]
AoA_list

,Word,AoA_Kup_lem
0,a,2.893384
1,aardvark,9.890000
2,abacus,8.690000
3,abacuses,8.690000
4,abalone,12.230000
...,...,...
51710,zucchini,6.790000
51711,zucchinis,6.790000
51712,zwieback,16.100000
51713,zygote,15.380000


In [11]:
WF_list = pd.read_excel('wordFrequency.xlsx', '3 wordForms')
WF_list

,lemRank,lemma,PoS,lemFreq,wordFreq,word
0,1,the,a,50033612,50033323,the
1,1,the,a,50033612,287,ze
2,2,be,v,32394756,10093608,is
3,2,be,v,32394756,6848519,was
4,2,be,v,32394756,6303682,'s
...,...,...,...,...,...,...
11455,5048,wheat,n,11877,11769,wheat
11456,5048,wheat,n,11877,108,wheats
11457,5049,predator,n,11876,6799,predators
11458,5049,predator,n,11876,5077,predator


In [12]:
stoplist = stopwords.words('english') # remember - they are lowercase
stoplist

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [13]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [14]:
allowed_chars = string.ascii_lowercase + "'" + "-" + "." + "/"
allowed_chars

"abcdefghijklmnopqrstuvwxyz'-./"

In [15]:
spell = Speller(lang='en')

In [16]:
def tokenize_and_preprocess(utt, stopwords=False):
    res = []
    contractions_removed = []
    for word in utt.split():
        contractions_removed.append(contractions.fix(word))
    contractions_removed = ' '.join(contractions_removed)

    for el in contractions_removed:
        if el in list(punctuation):
            contractions_removed = contractions_removed.replace(el, " ")

    for token in word_tokenize(contractions_removed):
        token = token.lower()
        if stopwords:
            if token not in set(stoplist + list(punctuation)) and\
                re.search('[a-zA-Z]', token) and\
                all(char in allowed_chars for char in token):
                token = spell(token)
                res.append(token)
        else:
            if token not in list(punctuation) and\
                re.search('[a-zA-Z]', token) and\
                all(char in allowed_chars for char in token):
                token = spell(token)
                res.append(token)
    return res
[tokenize_and_preprocess(chatC1['anonymised'][i], False) for i in range(len(chatC1['anonymised']))]


SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (4280761989.py, line 32)

In [ ]:
' '.join(['it', 's', 'true', 'that', 'they', 'are', 'still', 'there', 'now', 'so', 'present', 'simple', 'ok', 'but', 'especially', 'because', 'you', 'were', 'not', 'just', 'saying', 'that', 'but', 'you', 'were', 'saying', 'that', 'you', 'could', 'not', 'understand', 'them', 'so', 'this', 'is', 'more', 'specific', 'and', 'means', 'the', 'past', 'simple', 'would', 'be', 'better', 'really', 'by', 'the', 'way'])

In [ ]:
## Get chat's Age of Acquisition values
## return value: tuple(total, teacher, student)

def chat_AoA(chat):
    aoa_chat_total, aoa_chat_teacher, aoa_chat_student = 0, 0, 0

    for index, utt in chat.iterrows():
        utterance = utt['anonymised']
        role = utt['role']
        aoa_utt = 0
        aoa_n_tokens = 0

        utterance = utterance.replace('<STUDENT>', '')
        utterance = utterance.replace('<TEACHER>', '')
        tokens = tokenize_and_preprocess(utterance, True)

        for token in tokens:
            if token in AoA_list['Word'].values:
                aoa_utt += AoA_list[AoA_list['Word'] == token]['AoA_Kup_lem'].item()
                aoa_n_tokens += 1
        aoa_utt = aoa_utt/aoa_n_tokens if aoa_n_tokens > 0 else aoa_utt
        aoa_chat_total += aoa_utt

        if role == 'teacher':
            aoa_chat_teacher += aoa_utt
        elif role == 'student':
            aoa_chat_student += aoa_utt

    aoa_chat_total /= len(chat['anonymised'])
    aoa_chat_student /= len(chat[chat['role'].str.contains('student')])
    aoa_chat_teacher /= len(chat[chat['role'].str.contains('teacher')])

    return aoa_chat_total, aoa_chat_teacher, aoa_chat_student

In [ ]:
## Get chat's MLUs
## return value: tuple(total, teacher, student)

def chat_MLU(chat):
    teacher_utt = chat[chat['role'].str.contains('teacher')]
    student_utt = chat[chat['role'].str.contains('student')]

    return (
        np.mean(chat['nWords']),
        np.mean(teacher_utt['nWords']),
        np.mean(student_utt['nWords'])
    )

In [ ]:
WF_list_billion = pd.read_csv('unigram_freq.csv/unigram_freq.csv')
WF_list_billion

In [ ]:
## Get chat's Word Frequency values
## return value: tuple(total, teacher, student, rare_total, rare_teacher, rare_student)

def chat_WF(chat):
    wf_chat_total, wf_chat_teacher, wf_chat_student = 0, 0, 0
    tokens_total, tokens_teacher, tokens_student = 0, 0, 0
    rare_total, rare_teacher, rare_student = 0, 0, 0
    for index, utt in chat.iterrows():
        utterance = utt['anonymised']
        role = utt['role']
        wf_utt = 0
        wf_n_tokens = 0

        utterance = utterance.replace('<STUDENT>', '')
        utterance = utterance.replace('<TEACHER>', '')
        tokens = tokenize_and_preprocess(utterance, True)

        tokens_total += len(tokens)


        for token in tokens:
            if token in WF_list_billion['word'].values:
                try:
                    index = WF_list_billion.index[WF_list_billion['word'] == token].tolist()[0]
                except:
                    print(token)
                wf_utt += index
                wf_n_tokens += 1
            else:
                rare_total += 1
                if role == 'teacher':
                    rare_teacher += 1
                elif role == 'student':
                    rare_student += 1
        wf_utt = wf_utt/wf_n_tokens if wf_n_tokens > 0 else wf_utt
        wf_chat_total += wf_utt

        if role == 'teacher':
            wf_chat_teacher += wf_utt
            tokens_teacher += len(tokens)
        elif role == 'student':
            wf_chat_student += wf_utt
            tokens_student += len(tokens)

    wf_chat_total /= len(chat['anonymised'])
    wf_chat_student /= len(chat[chat['role'].str.contains('student')])
    wf_chat_teacher /= len(chat[chat['role'].str.contains('teacher')])
    return wf_chat_total, wf_chat_teacher, wf_chat_student, rare_total/tokens_total, rare_teacher/tokens_teacher, rare_student/tokens_student

chat_WF(chatC1)

In [ ]:
'''This is for the COCA corpus'''

# ## Get chat's Word Frequency values
# ## return value: tuple(total, teacher, student, rare_total, rare_teacher, rare_student)
#
# def chat_WF(chat):
#     wf_chat_total, wf_chat_teacher, wf_chat_student = 0, 0, 0
#     tokens_total, tokens_teacher, tokens_student = 0, 0, 0
#     rare_total, rare_teacher, rare_student = 0, 0, 0
#     for index, utt in chat.iterrows():
#         utterance = utt['anonymised']
#         role = utt['role']
#         wf_utt = 0
#         wf_n_tokens = 0
#
#         utterance = utterance.replace('<STUDENT>', '')
#         utterance = utterance.replace('<TEACHER>', '')
#         tokens = word_tokenize(utterance)
#
#         # we get rid of the stoplist words
#         tokens = [token.lower() for token in tokens if token.lower() not in set(stoplist+list(punctuation)) and re.search('[a-zA-Z]', token) and all(char in string.ascii_lowercase + "'" + "-" + "." + "/" for char in token.lower())]
#
#         tokens_total += len(tokens)
#
#         predicted_tokens_classes = nltk.pos_tag(tokens)
#
#         for token, prediction in zip(tokens, predicted_tokens_classes):
#             if token in WF_list['word'].values:
#                 # print(token)
#                 predicted_tag = prediction[1][0].lower()
#                 # if len(WF_list.loc[WF_list['word'] == token]) == 1:
#                 #     list_entry = WF_list.loc[WF_list['word'] == token]
#                 # else:
#                 #     list_entry =
#                 list_entry = WF_list.loc[WF_list['word'] == token]
#                 if len(list_entry) > 1:  # if more then one option, use the predicted tag
#                     list_entry = list_entry[list_entry['PoS'] == predicted_tag]
#                     if len(list_entry) > 1:
#                         # specifically for "found" which returns two words with the same tag
#                         if prediction == 'VBN':
#                             list_entry = list_entry.head(1) # past form of 'find'
#                         else:
#                             list_entry = list_entry.iloc[[1]] # the verb 'found'
#                 if len(list_entry) == 0:  # if the predicted tag is wrong, use the most frequent version
#                     list_entry = WF_list.loc[WF_list['word'] == token].head(1)
#                 # print(token)
#                 # print(list_entry['lemFreq'].item())
#                 try:
#                     wf_utt += list_entry['lemRank'].item()
#                 except ValueError:
#                     print(f"Token: {token}\nNLTK Prediction: {prediction}\nList entry: {list_entry}\nList entry before: {WF_list.loc[WF_list['word'] == token]}\nList entry top: {WF_list.loc[WF_list['word'] == token].head(1)}")
#
#                 wf_n_tokens += 1
#             else:
#                 rare_total += 1
#                 # print(token, 'not in list')
#                 if role == 'teacher':
#                     rare_teacher += 1
#                 elif role == 'student':
#                     rare_student += 1
#         wf_utt = wf_utt/wf_n_tokens if wf_n_tokens > 0 else wf_utt
#         wf_chat_total += wf_utt
#
#         if role == 'teacher':
#             wf_chat_teacher += wf_utt
#             tokens_teacher += len(tokens)
#         elif role == 'student':
#             wf_chat_student += wf_utt
#             tokens_student += len(tokens)
#
#     wf_chat_total /= len(chat['anonymised'])
#     wf_chat_student /= len(chat[chat['role'].str.contains('student')])
#     wf_chat_teacher /= len(chat[chat['role'].str.contains('teacher')])
#     return wf_chat_total, wf_chat_teacher, wf_chat_student, rare_total/tokens_total, rare_teacher/tokens_teacher, rare_student/tokens_student
#
# chat_WF(chatC1)

In [ ]:
## Get chat's Type-Token ratio values
## return value: tuple(total, teacher, student)

def chat_TTR(chat):
    token_list_student = []
    token_list_teacher = []

    for index, utt in chat.iterrows():
        utterance = utt['anonymised']
        role = utt['role']

        utterance = utterance.replace('<STUDENT>', '')
        utterance = utterance.replace('<TEACHER>', '')

        word_tokens = tokenize_and_preprocess(utterance, False)
        if role == 'teacher':
            token_list_teacher += word_tokens
        elif role == 'student':
            token_list_student += word_tokens

    return (
        (len(set(token_list_teacher)) + len(set(token_list_student)))/(len(token_list_teacher) + len(token_list_student)),
        len(set(token_list_teacher))/len(token_list_teacher),
        len(set(token_list_student))/len(token_list_student)
    )

In [ ]:
def chat_utterance_proportion(chat):
    return len(chat[chat['role'].str.contains('student')])/len(chat)

In [ ]:
def chat_words_proportion(chat):
    return np.sum((chat[chat['role'].str.contains('student')]['nWords']))/np.sum(chat['nWords'])

In [ ]:
def get_student_timegaps(student, contents):
    first_class_time = datetime.strptime(contents['start.time'].iloc[0], "%Y-%m-%d %H:%M:%S")
    last = first_class_time
    gaps = []
    for class_time in contents['start.time'][1:]:
        date_class_time = datetime.strptime(class_time, "%Y-%m-%d %H:%M:%S")
        gaps.append((date_class_time-last).days)
        last = date_class_time
    return gaps

In [ ]:
## Now, I analyse progress of particular students

In [ ]:
def calc_scaffolding_proportion(chat):
    chat_dropped = chat.dropna(subset=['seq.type'])
    chat_dropped = chat_dropped[chat_dropped['role'].str.contains('teacher')]

    return len(chat_dropped[chat_dropped['seq.type'].str.contains('scaffolding')])/len(chat[chat['role'].str.contains('teacher')])

calc_scaffolding_proportion(chatC1)

In [ ]:
students_progress = {}

In [ ]:
students_progress = json.load(open('progress_students.txt'))
students_progress

In [ ]:
chatrooms_dropped = chatrooms_all.dropna(subset=['seq.type'])

scaffolding_utts = chatrooms_dropped[chatrooms_dropped['seq.type'].str.contains('scaffolding')]
scaffolding_utts

In [ ]:
preceding_turns_scaff = chatrooms_all.iloc[scaffolding_utts.index-1]

In [ ]:
def chat_flesch_kincaid(chat):
    teacher_all, student_all = "", ""
    for index, utt in chat.iterrows():
        utterance = utt['anonymised']
        if len(word_tokenize(utterance)) < 3:  # Discarding utts like "ok", "hi"
            continue
        role = utt['role']

        utterance = utterance.replace('<STUDENT>', '')
        utterance = utterance.replace('<TEACHER>', '')
        if utterance[-1:] not in ('.', '!', '?'):
            utterance += '.'  # finish all utts in full stops

        if role == 'teacher':
            teacher_all += ' ' + utterance
        elif role == 'student':
            student_all += ' ' + utterance

    fk_total = Readability(teacher_all+student_all).flesch_kincaid().score if len(tokenize_and_preprocess(teacher_all+student_all)) >= 100 else 0
    fk_teacher = Readability(teacher_all).flesch_kincaid().score if len(tokenize_and_preprocess(teacher_all)) >= 100 else 0
    fk_student = Readability(student_all).flesch_kincaid().score if len(tokenize_and_preprocess(student_all)) >= 100 else 0

    return fk_total, fk_teacher, fk_student
chat_flesch_kincaid(chatC1)

In [ ]:
def chat_flesch(chat):
    teacher_all, student_all = "", ""
    for index, utt in chat.iterrows():
        utterance = utt['anonymised']
        if len(word_tokenize(utterance)) < 3:  # Discarding utts like "ok", "hi"
            continue
        role = utt['role']

        utterance = utterance.replace('<STUDENT>', '')
        utterance = utterance.replace('<TEACHER>', '')
        if utterance[-1:] not in ('.', '!', '?'):
            utterance += '.'  # finish all utts in full stops
        # print(utterance)

        if role == 'teacher':
            teacher_all += ' ' + utterance
        elif role == 'student':
            student_all += ' ' + utterance

    # print(student_all)
    flesch_total = Readability(teacher_all+student_all).flesch().score if len(tokenize_and_preprocess(teacher_all+student_all)) >= 100 else 0
    flesch_teacher = Readability(teacher_all).flesch().score if len(tokenize_and_preprocess(teacher_all)) >= 100 else 0
    flesch_student = Readability(student_all).flesch().score if len(tokenize_and_preprocess(student_all)) >= 100 else 0

    return flesch_total, flesch_teacher, flesch_student
chat_flesch(chatC1)

In [ ]:
def chat_gunning_fog(chat):
    teacher_all, student_all = "", ""
    for index, utt in chat.iterrows():
        utterance = utt['anonymised']
        if len(word_tokenize(utterance)) < 3:  # Discarding utts like "ok", "hi"
            continue
        role = utt['role']

        utterance = utterance.replace('<STUDENT>', '')
        utterance = utterance.replace('<TEACHER>', '')
        if utterance[-1:] not in ('.', '!', '?'):
            utterance += '.'  # finish all utts in full stops
        # print(utterance)

        if role == 'teacher':
            teacher_all += ' ' + utterance
        elif role == 'student':
            student_all += ' ' + utterance

    # print(student_all)
    gf_total = Readability(teacher_all+student_all).gunning_fog().score if len(tokenize_and_preprocess(teacher_all+student_all)) >= 100 else 0
    gf_teacher = Readability(teacher_all).gunning_fog().score if len(tokenize_and_preprocess(teacher_all)) >= 100 else 0
    gf_student = Readability(student_all).gunning_fog().score if len(tokenize_and_preprocess(student_all)) >= 100 else 0

    return gf_total, gf_teacher, gf_student
chat_gunning_fog(chatC1)

In [ ]:
def get_turns(chat):
    turns = {
        'turn': [],
        'speaker': []
    }
    curr_turn = ""
    curr_speaker = chat['role'][0]
    for index, utt in chat.iterrows():
        if utt['role'] != curr_speaker:
            turns['turn'].append(curr_turn)
            turns['speaker'].append(curr_speaker)
            curr_turn = ""  # reset when new speaker is speaking

        curr_turn += utt['anonymised']
        if curr_turn[-1] not in ('.', '!', '?'):
            curr_turn += '.'
        curr_turn += ' '

        curr_speaker = utt['role']

    # add the last utterance
    turns['turn'].append(curr_turn)
    turns['speaker'].append(curr_speaker)   
    
    return pd.DataFrame.from_dict(turns)

In [ ]:
get_turns(chatC1)[69:71]

In [ ]:
ex_utt = chatC1.loc[94:96, ['anonymised', 'role']]
ex_utt.columns = ['utterance','speaker']
ex_utt

In [ ]:
import math
def teacher_student_utt_pairs(chat):
    teacher_utts = chat.loc[chat['role'] == 'teacher']
    teacher_utts = teacher_utts.drop(teacher_utts[teacher_utts['turn.number'] == 1].index)

    previous_utts = pd.DataFrame(columns=list(teacher_utts.columns))
    for index, utt in teacher_utts.iterrows():
        if isinstance((utt['responding.to']), float): # still wrong
            if not math.isnan(utt['responding.to']):
                # print('res')
                previous_utts = pd.concat(
                    [previous_utts, pd.DataFrame(chat.loc[chat['turn.number'] == utt['responding.to']])]
                )
            else:
                turn_number = int(utt['turn.number'])-1
                # print(turn_number)
                while turn_number > 0 and turn_number in chat['turn.number'].values and not chat.loc[chat['turn.number'] == turn_number]['role'].values[0] == 'student':
                    turn_number -= 1
                if turn_number == 0:
                    continue
                previous_utts = pd.concat(
                    [previous_utts, pd.DataFrame(chat.loc[chat['turn.number'] == turn_number])]
                )

    return teacher_utts, previous_utts

In [ ]:
teacher_student_utt_pairs(chatC1)[0].shape

In [ ]:
teacher_student_utt_pairs(chatC1)[1].shape

In [ ]:
def total_utt_pairs(chat):
    current_utts = chat[1:] # discarding the first utt
    previous_utts = pd.DataFrame(columns=list(chat.columns))

    for index, utt in current_utts.iterrows():
        role = utt['role']
        if isinstance(utt['responding.to'], float):
            if not math.isnan(utt['responding.to']):
                # print('res')
                previous_utts = pd.concat(
                    [previous_utts, pd.DataFrame(chat.loc[chat['turn.number'] == utt['responding.to']])]
                )
            else:
                turn_number = int(utt['turn.number'])-1
                # print(turn_number)
                while turn_number > 0 and turn_number in chat['turn.number'].values and chat.loc[chat['turn.number'] == turn_number]['role'].values[0] == role:
                    turn_number -= 1
                if turn_number == 0:
                    continue
                previous_utts = pd.concat(
                    [previous_utts, pd.DataFrame(chat.loc[chat['turn.number'] == turn_number])]
                )

    return current_utts, previous_utts

In [ ]:
isinstance((chatC1['responding.to'].values[0]), float)

In [ ]:
chatC1['responding.to'].values[0]

In [ ]:
total_utt_pairs(chatC1)[0].shape

In [ ]:
total_utt_pairs(chatC1)[1].shape

In [ ]:
def tutor_overlaps_chat(chat):
    overlaps = []
    teacher_utts, previous_utts = teacher_student_utt_pairs(chat)

    for (utt, previous) in zip(teacher_utts['anonymised'], previous_utts['anonymised']):
        tokens = tokenize_and_preprocess(utt, True)
        previous_tokens = tokenize_and_preprocess(previous, True)

        intersect = set(previous_tokens).intersection(set(tokens))
        union = set(previous_tokens).union(set(tokens))
        
        if len(union) > 0:
            overlaps.append(len(intersect)/len(union))  # jaccard

    return np.mean(overlaps)

In [ ]:
tutor_overlaps_chat(chatC1)

In [ ]:
def total_overlaps_chat(chat):
    overlaps = []
    current_utts, previous_utts = total_utt_pairs(chat)

    for (utt, previous) in zip(current_utts['anonymised'], previous_utts['anonymised']):
        tokens = tokenize_and_preprocess(utt, True)
        previous_tokens = tokenize_and_preprocess(previous, True)
        
        intersect = set(previous_tokens).intersection(set(tokens))
        union = set(previous_tokens).union(set(tokens))
        
        if len(union) > 0:
            overlaps.append(len(intersect)/len(union))  # jaccard

    return np.mean(overlaps)

In [ ]:
total_overlaps_chat(chatC1)

In [ ]:
def tutor_overlaps_turns_chat(chat):
    overlaps = []
    chat_turns = get_turns(chat)
    prev_student = ''
    for index, row in chat_turns.iterrows():
        if row['speaker'] == 'student':
            prev_student = row['turn']
        elif prev_student:
            student_tokens = set(tokenize_and_preprocess(prev_student, True))
            teacher_tokens = set(tokenize_and_preprocess(row['turn'], True))
            
            intersect = student_tokens.intersection(teacher_tokens)
            union = student_tokens.union(teacher_tokens)
            
            if len(union) > 0:
                overlaps.append(len(intersect)/len(union))  # jaccard

    return np.mean(overlaps)

In [ ]:
tutor_overlaps_turns_chat(chatC1)

In [ ]:
def either_overlaps_turns_chat(chat):
    overlaps = []
    chat_turns = get_turns(chat)
    prev_turn = chat_turns['turn'][0]
    for index, row in chat_turns[1:].iterrows():
        prev_tokens = set(tokenize_and_preprocess(prev_turn, True))
        curr_tokens = set(tokenize_and_preprocess(row['turn'], True))

        intersect = prev_tokens.intersection(curr_tokens)
        union = prev_tokens.union(curr_tokens)
        
        if len(union) > 0:
            overlaps.append(len(intersect)/len(union))  # jaccard
            
        prev_turn = row['turn']

    return np.mean(overlaps)

In [ ]:
either_overlaps_turns_chat(chatC1)

In [ ]:
def pos_overlap_chat(chat):  # how many pos were repeated from the prev utterance
    overlaps = []
    chat_turns = get_turns(chat)
    prev_turn = chat_turns['turn'][0]
    for index, row in chat_turns[1:].iterrows():
        prev_tokens = tokenize_and_preprocess(prev_turn)
        curr_tokens = tokenize_and_preprocess(row['turn'])
        
        prev_pos = set([tag for word, tag in nltk.pos_tag(prev_tokens)])
        curr_pos = set([tag for word, tag in nltk.pos_tag(curr_tokens)])

        intersect = prev_pos.intersection(curr_pos)
        union = prev_pos.union(curr_pos)
        
        if len(union) > 0:
            overlaps.append(len(intersect)/len(union))  # jaccard
            
        prev_turn = row['turn']

    return np.mean(overlaps)

In [ ]:
pos_overlap_chat(chatC1)

It only takes into account how similar the tutor's and teacher's grades are

In [ ]:
def get_complexity_matching_MLU(chat):
    mlu_scores = chat_MLU(chat)
    diff = mlu_scores[1]-mlu_scores[2]
    return diff

In [ ]:
get_complexity_matching_MLU(chatrooms[0])

In [ ]:
def get_complexity_matching_AoA(chat):
    AoA_scores = chat_AoA(chat)
    diff = AoA_scores[1]-AoA_scores[2]
    return diff

In [ ]:
get_complexity_matching_AoA(chatrooms[0])